**Lab 3: Geopandas and Rasterio**



---





1: Loading the Data

 I mounted my google drive to my google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # mount the content from google drive into the google colab notebook

2: Importing Libraries

I imported the necessary python libraries to aid my analysis

In [ ]:
pip install contextily # Install contextily library


In [ ]:
pip install mapclassify # Install mapclassify library

In [ ]:
# import libraries for analysis

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import contextily as ctx
import rasterio as rio
from rasterio import plot

plt.rcParams['figure.figsize'] = [10, 8]

3: Reading in the data

I read in the data using geopandas

In [ ]:
gdf = gpd.read_file("/content/drive/My Drive/Census20_LSOA.shp") # loads in the shapefile data using geopandas
gdf.head()  # shows 1st 5 rows of the dataset

4: Subsetting

I subsetted the following columns, and geometry

LSOA11CD --> LSOA area code LSOA11NM --> LSOA area name LSOA11NMW --> LSOA bigger area Pop20 --> Population counts

In [ ]:
subset = gdf[["LSOA11CD","LSOA11NM","LSOA11NMW","Pop20","geometry"]]  # subsets these columns for analysis and adds the geometries
subset.head() #shows 1st 5 rows of dataset

5: Find CRS

I identified the coordinate reference system of the layer

In [ ]:
print(gdf.crs) # shows coordinate reference system of layer

6: How many Features

I identified the number of features in the layer and expressed the results as an integer

In [ ]:
feature_numbers = int(len(gdf))  # counts number of features in dataframe
print(feature_numbers)  # prints results

7: Check for Duplicates

I made sure that the values in the "LSOA11CD" column were unique by using pandas to check for duplicates

In [ ]:
duplicates = gdf.duplicated('LSOA11CD')  # checks for duplicates in the "LSOA11CD" column
print(duplicates)  # prints results (true or false)

8: Plotting

I plot the layer using the .plot method

In [ ]:
gdf.plot()  # Plots dataset for visualisation

9: Plotting 2.0

I plot the layer using the .explorer method

In [ ]:
gdf.explore()  # maps dataset in dynamic manner

10: Subsetting

I subset just the LSOA areas with Pop20 counts greater than 1500.

In [ ]:
population_subset = gdf.mask(gdf["Pop20"] <= 1500).dropna(subset=["Pop20"])  # removes population counts the same as or lower than 1500
print(population_subset)  # prints results

11: Plotting the subset

I plotted the resulting subset, using symbology according to total population size, i.e., the "Pop_Total" column, and using a sequentual color map such as "Reds"

In [ ]:
gdf.plot(column="Pop20", cmap="Reds", legend=True, scheme="quantiles", figsize=(10, 8))  # plots population data in red, using qualtile scheme

12: How many Areas

I identified how many areas there were in the requested population using the shape function

In [ ]:
print(population_subset.shape[0])  # shows the number of areas in the subset

13: What's the Population

I identified the total population of the subset layer using the sum function

In [ ]:
total_pop = population_subset["Pop20"].sum()  # calculates total population in the population subset
print(total_pop) # prints results

**Rasterio** **Exercises**


---





1: Import Rasterio

I imported the python library rasterio to aid my analysis

In [ ]:
import rasterio as rio  # imports library

2: Reading in the file

I read in the file as a rasterio dataset using rio.open

In [ ]:
dataset = rio.open("/content/drive/My Drive/Clipped_Raster.tif")  # loads the tif file from google drive

3: Check CRS

I identified the crs of the dataset

In [ ]:
print(dataset.crs)  # shows coordinate reference system of the dataset

4: Extent

I identified the raster extent of the dataset in projected coordinates

In [ ]:
print(dataset.bounds)  # shows extent of the dataset

5: How many Bands

I identified how many bands there were in the dataset

In [ ]:
print(dataset.count)  # shows number of bands in dataset

6: Plot

I created a plot of the image

In [ ]:
plt.rcParams['figure.figsize'] = [10,8]  # input figure parameters
plt.imshow(dataset.read(1), cmap='gray')  # shows raster image in greyscale

7: Histogram

I created a histogram from the raster

In [ ]:
plt.hist(dataset.read(1).flatten(), bins='auto')  # creates histogram of raster with automatic bin sizing

8: False colour plot

I created a false colour plot of the raster using the python library EarthPy

In [ ]:
band1 = dataset.read(1)
ep.plot_rgb(band1,
            rgb=[3, 2, 1],
            stretch=True,
            str_clip=10)
plt.show()                     # This code failed to run because false colour plots require multiple bands and this dataset only has 1 band

**Lab 4: Spatial Clustering (k-means and DBSCAN)**

---



**Part A: Data Exploration and Pre-Proccessing**

---



1: Loading in the data

I mounted my google drive to my google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # mounts content from google drive to google colab notebook

2: Importing Libraries

I imported the necessary puthon libraries to aid my analysis

In [ ]:
pip install lonboard  # imports library

In [ ]:
# Imports necessary library for analysis

import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
import shapely
import folium
import seaborn as sns
from lonboard import Map, ScatterplotLayer, viz

3: Reading in the data
I used pandas to load in the car accidents data

In [ ]:
car_accidents = pd.read_csv('/content/drive/MyDrive/UK_Accident.csv')  # loads csv file from google drive

4: Display top of dataset

I I displayed the 1st few rows of the dataset to see the available attributes

In [ ]:
print(car_accidents.head())  # shows 1st 5 rows of the dataset

5: Necessary Columns

I kept the necessary columns from the dataset so that the datset had less noise

In [ ]:
necessary_columns = ["Accident_Severity", "Number_of_Vehicles", "Number_of_Casualties", "Speed_limit", "Day_of_Week", "Road_Type", "Light_Conditions", "Weather_Conditions", "Road_Surface_Conditions", "Urban_or_Rural_Area", "Year", "Latitude", "Longitude"]
car_accidents = car_accidents[necessary_columns]  # selects necessary columns into a subset of the data
print(car_accidents.head())  # we can see the change now

6: Slicing the Dataframe

I sliced the pandas dataframe until I was left with only 2010 records in order to specify and reduce the analysis

In [ ]:
car_accidents_2010 = car_accidents[car_accidents["Year"] == 2010]  # slices dataframe into a subset with only 2010 records
print(car_accidents_2010.head())  # we can see the change now

7: Plotting

I used the dataset to make a plot showing which day of the week had the most car accidents

In [ ]:
day_num = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"  # turn numerical values into days for the plot
}
car_accidents_2010["Day_of_Week"] = car_accidents_2010["Day_of_Week"].map(day_num)
accidents_per_day = car_accidents_2010["Day_of_Week"].value_counts()  # counts accidents per day
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]  # creates the order for the graph
accidents_per_day = accidents_per_day.reindex(day_order)  # reorders from monday to Sunday
accidents_per_day.plot(kind="bar")  # plots the graph

8: Plotting a Relationship

I made a plot to explore the relationship between Accident Severity and Road Conditions

In [ ]:
accident_severity_road_condition = pd.crosstab(  # creates contingency table for attributes
    car_accidents_2010["Accident_Severity"],
    car_accidents_2010["Road_Surface_Conditions"]
)
accident_severity_road_condition.plot(kind="bar")  # graphs contingency table as bar graph showing relationship

We can see that wet/damp roads make up a good chunk of the accidents within severity level 3. However severity level 3 is dominated by dry road conditions. Intuitively this might seem unusual but once you realise that dry roads are most commonly driven on, it makes more sense. In this instance the graph casn be misleading as it's not ratiod down with other road conditions

8: Mappin car accidents

I used the lonboard python library to map all the car accidents within my filtered 2010 dataset

In [ ]:
from matplotlib.artist import get
import lonboard as lb  # imports libraries

geometry = gpd.points_from_xy(car_accidents_2010["Longitude"], car_accidents_2010["Latitude"])  # changes latitude and longitude into points

gdf_accidents = gpd.GeoDataFrame(car_accidents_2010, geometry=geometry, crs="EPSG:4326")  # geo data frame made from geometry points and using "EPSG:4326" as the CRS



lb.viz(gdf_accidents)  # maps the points for accidents

9: Spatial Filtering

I I created a new dataset  to map only the car accidents in the Glasgow-Edinburgh region. I then created another map using the lonboard library to display the car accidents only in that region

In [ ]:
from shapely.geometry import box  # imports library


ge_bbox = [-4.46, 55.72, -3.00, 55.97]  # defines bounding box
gdf_accidents_ge = gdf_accidents[gdf_accidents.intersects(box(*ge_bbox))]  # flters accidents that intersect within this bounding box
lb.viz(gdf_accidents_ge)  # maps the accident points

**Part B: K-means Clustering Implementation**

---



1: Implement Kmeans clustering

I implemented K-means clustering with different values of k to the filtered dataset that I created for the Glasgow-Edinburgh region.

In [ ]:
X = gdf_accidents_ge[['Longitude', 'Latitude']]  # X becomes the latitude and longitude values for the geo data frame
kmeans3 = KMeans(n_clusters=3, random_state=42)
gdf_accidents_ge["cluster_3"] = kmeans3.fit_predict(X)  # implements kmeans clustering for 3 clusters

kmeans5 = KMeans(n_clusters=5, random_state=42)
gdf_accidents_ge["cluster_5"] = kmeans5.fit_predict(X)  # implements kmeans clustering for 3 clusters

2: Mapping Clusters

I mapped the clusters using the lonboard library

In [ ]:
from lonboard import Map, ScatterplotLayer, viz  # imports libraries

lb.viz(gdf_accidents_ge[["geometry", "cluster_3"]])  # maps 3 clusters for the geo data frame

In [ ]:
lb.viz(gdf_accidents_ge[["geometry", "cluster_5"]])  # maps 5 clusters for the geo data frame

3: Describe the Clustering Results


As k determines the amount of clusters that will be produced, an increase in k add more detail to the cluster map. For example changing k from 3 to 5 will show more specified correlations as it's essentially finding neighbouring points on a smaller scale.

4: Kmeans Clustering with Attributes

I implemented kmeans clustering again but this time I included the data frames "Accident_Severity" and "Number_of_Vehicles

In [ ]:
X_Factors = gdf_accidents_ge[["Accident_Severity", "Number_of_Vehicles"]]  # inputs attributes for clustering

kmeans3_factors = KMeans(n_clusters=3, random_state=42)
gdf_accidents_ge["cluster_3_factors"] = kmeans3_factors.fit_predict(X_Factors)  # implements kmeans clustering for 3 clusters based on the attributes provided

kmeans5_factors = KMeans(n_clusters=5, random_state=42)
gdf_accidents_ge["cluster_5_factors"] = kmeans5_factors.fit_predict(X_Factors)  # implements kmeans clustering for 5 clusters based on the attributes provided

5: Visualise the results

I mapped the results using the lonboard library

In [ ]:
from lonboard import Map, ScatterplotLayer, viz  # imports library

lb.viz(gdf_accidents_ge[["geometry", "cluster_3_factors"]])  # maps the accident points for 3 clusters in the geo data frame based on the attributes provided

In [ ]:
from lonboard import Map, ScatterplotLayer, viz  # imports library

lb.viz(gdf_accidents_ge[["geometry", "cluster_5_factors"]])  # maps the accident points for 5 clusters in the geo data frame based on the attributes provided

6: Comparison between coordinate and attribute clusters

As previously stated, the cluster map with only coordinates is showing spatial patterns about where accidents are taking place. On the other hand, the cluster map that includes attributes is showing spatial patterns of what kinds of accidents are happening where. So by increasing k for the cluster map with attributes, in detail clustering of accident type can allow for analysis as to why certain accident types are occuring in a specific region

**Part C: Spatial Analysis and DBSCAN Clustering (Spatial Correlation)**

---



1: Create new geopandas dataframe

I created another GeoPandas Dataframe by rereading the data to avoid any confusion with the previous geodataframe. I used this dataframe to begin with DBSCAN clustering

In [ ]:
car_accidents = pd.read_csv("/content/drive/MyDrive/UK_Accident.csv")  # loads csv file using pandas
necessary_columns = ["Accident_Severity", "Number_of_Vehicles", "Number_of_Casualties", "Speed_limit", "Day_of_Week", "Road_Type", "Light_Conditions", "Weather_Conditions", "Road_Surface_Conditions", "Urban_or_Rural_Area", "Year", "Latitude", "Longitude"]
car_accidents = car_accidents[necessary_columns]  # filters the selected columns into a subset of the dataset

geometry = gpd.points_from_xy(car_accidents["Longitude"], car_accidents["Latitude"])  # changes latitude and longitude into geometry points
gdf_accidents_dbscan = gpd.GeoDataFrame(car_accidents, geometry=geometry, crs="EPSG:4326")  # geo data frame made from geometry points and using "EPSG:4326" as the CRS
print(gdf_accidents_dbscan.head())  # shows 1st 5 rows in the dataset

2: Find Necessary coordinates

I used the BBox website to filter the new geodataframe to contain only the accidents around Birmingham

In [ ]:
from shapely.geometry import box  # imports library


bh_bbox = [-2.05, 52.35, -1.75, 52.55]  # defines bounding box
gdf_accidents_bh = gdf_accidents[gdf_accidents.intersects(box(*bh_bbox))]  # filters accidents that intersect within this bounding box

3: Map the dataset

I used the lonboard library to map the Birmingham dataset

In [ ]:
lb.viz(gdf_accidents_bh)  # maps accident points for the geo data frame

4: Identify datatypes in the dataset

I used .dtypes to see which attributes were categorical and which ones were numerical which helped with calculating correlations

In [ ]:
print(gdf_accidents_bh.dtypes)  # shows data types of all the attributes

5: Correlating Numerical Attributes

I ran a correlation for the numerical attributes in my dataset

In [ ]:
corr = gdf_accidents_bh.corr(numeric_only=True)  # runs a correlation, but only for numerical attributes
print(corr)  # prints the correlation results

6: Correlation Heatmap

I adjusted a given code by adding in my dataset to create a heatmap of correlation values for the numerical attributes. The code is as follows...

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr,
    annot=True,
    cmap='coolwarm',
    fmt='.2f',           # Format annotations to 2 decimal places
    linewidths=0.5,
    cbar_kws={'label': 'Correlation Coefficient'}
)

plt.title('Pearson -Correlation Matrix')
plt.show()

7: Install Libraries

I installed the pysal library to aid my analysis

In [ ]:
pip install pysal  # import library

8: Install more Libraries

I imported additional libraries to aid my analysis

In [ ]:
import necessary libraries for analysis

import libpysal.weights as weights
from esda.moran import Moran

9: Reprojecting the dataset

I reprojected the dataset so that it alligned with the UK for spatial analysis

In [ ]:
gdf_accidents_bh = gdf_accidents_bh.to_crs(epsg=27700)  # converts crs of geo data frame into epsg=27700
print(gdf_accidents_bh.crs)  # we can see that it has now changed

10: Running Local Moran's I and Spatial clustering DBSCAN

I adjusted a given code by adding in my variables and datasets in order to find out Morans I statistic value and also p-values for statistical analysis. The code is as follows...

In [ ]:
w = weights.DistanceBand.from_dataframe(gdf_accidents_bh, threshold=500, ids=gdf_accidents_bh.index) #Adjust this line to match your variables.
w.transform = 'R'
moran = Moran(gdf_accidents_bh['Accident_Severity'], w) #Adjust this line to match your variables.

print(f"\n--- Moran's I Spatial Autocorrelation Analysis ---")
print(f"Defined {w.n} observations and {w.mean_neighbors:.2f} average neighbors per point.")
print(f"\nMoran's I Statistic (Observed I): {moran.I:.4f}")
print(f"P-value (significance): {moran.p_sim:.4f}")

11: Describe the Results of the Correlation Analysis


As Moran's I statistic value of -0.0057 is very close to zero, I think it can be concluded that accident severity follows a random spatial pattern. Also a p value of 0.2950 is more than the threshold of 0.05, therefore concluding that the results are not statistically significant. Ultimately, there is no spatial correlation for accident severity.

**Part D: DBSCAN Clustering Implementation**

---



1: Implementing DBSCAN Clustering

I implemented DBSCAN clustering with different eps and min_samples to the projected dataset.

In [ ]:
gdf_accidents_bh['X'] = gdf_accidents_bh.geometry.x  # turns geometry points into numerical values
gdf_accidents_bh['Y'] = gdf_accidents_bh.geometry.y

dbscan = DBSCAN(eps=500, min_samples=10)  # eps(neigbour radius at 500m) and min_samples is 10 (number of points within radius to identify it as a cluster)
gdf_accidents_bh["cluster_dbscan"] = dbscan.fit_predict(gdf_accidents_bh[['X', 'Y']])  # implements DBSCAN clustering for the latitude and longitude of the geo data frame

print(gdf_accidents_bh[['X', 'Y', 'cluster_dbscan']].head())  # shows DBSCAN clustering results

In [ ]:
dbscan = DBSCAN(eps=500, min_samples=2)  # decreasing min_samples will make it easier for clusters to form as threshold is lower
gdf_accidents_bh["cluster_dbscan"] = dbscan.fit_predict(gdf_accidents_bh[['X', 'Y']])

print(gdf_accidents_bh[['X', 'Y', 'cluster_dbscan']].head())

In [ ]:
dbscan = DBSCAN(eps=500, min_samples=5)  # changes in eps will either hone in on a cluster or demonstrate a large spatial pattern based on whether the neighbour radius is lower or higher
gdf_accidents_bh["cluster_dbscan"] = dbscan.fit_predict(gdf_accidents_bh[['X', 'Y']])

print(gdf_accidents_bh[['X', 'Y', 'cluster_dbscan']].head())

In [ ]:
dbscan = DBSCAN(eps=500, min_samples=10)
gdf_accidents_bh["cluster_dbscan"] = dbscan.fit_predict(gdf_accidents_bh[['X', 'Y']])

print(gdf_accidents_bh[['X', 'Y', 'cluster_dbscan']].head())

In [ ]:
dbscan = DBSCAN(eps=500, min_samples=20)
gdf_accidents_bh["cluster_dbscan"] = dbscan.fit_predict(gdf_accidents_bh[['X', 'Y']])

print(gdf_accidents_bh[['X', 'Y', 'cluster_dbscan']].head())

2: Mapping Clusters

I mapped the clusters using the plotly library

In [ ]:
import plotly.express as px  # imports library

fig_dbscan = px.scatter_mapbox(
    gdf_accidents_bh,  # geo data frame is the input for the map
    lat="Latitude",
    lon="Longitude",
    color="cluster_dbscan",  # colour points based of DBSCAN cluster assignment
    mapbox_style="carto-positron",
    zoom=9,
    title="DBSCAN Clustering",  # map title
    height=700,
    opacity=0.5,

)
fig_dbscan.show()

3: Describe the Culster Results

From different clustering results, I can see that more values are identified as noise when eps is lower because it means that less point meet neighbour requirement. Similarly, as min_samples increase, the number of values identified as noise increases as areas would need to be very dense in order for clusters to form.

4: Comparison between kmeans and DBSCAN clusters

The Kmeans clusters allowed me to preset the number of clusters that would created, therefore allowing me to control the analysis I was conducting. This in some way made it simpler and was good for seeing overall patterns. The DBSCAN clusters was good because density could be found without having to input the number of clusters. On the other hand, inputting eps and min_samples meant that certain parameters would work better than others and this could be a problem is certain parameters are left out.

5: What are the real-world implications of identified clusters in the field of urban planning?

In terms of road accidents, identifying clusters means that certain regions can be highlighted for investigation and lead to possible changes in infrastructure to try and decrease accident levels. Areas can therefore be prioritized based on the clusters and even though the reason for the accidents are not included, it gives a spatial overview as to where change is required.

**Final Challenge**

---



1: Importing Libraries

I imported the pandas library in order to aid my analysis

In [ ]:
import pandas as pd  # imporrts library

2: Loading the Data

I mounted my google drive to my google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')  # mount the content from my google drive into my google colab notebook

3: Reading in the Data

I read in the data using pandas

In [ ]:
df = pd.read_csv('/content/drive/My Drive/world_cities.csv')  # load the csv file using pandas

In [ ]:
df.head()  # shows the 1st 5 rows of the dataset

4: Create "pop_M" column

I created a new column called "pop_M" that showed the population in millions. To do so I divided the "pop" column by 1000000

In [ ]:
df["pop_M"] = df["pop"] / 1000000  # creates pop_M column by divided pop column by 1000000

In [ ]:
df.head()  # We can see that pop_M has been added

5: Remove "pop" column

I removed the population column from the dataset so that "pop_M" was left

In [ ]:
df = df.drop(columns=["pop"])  # removes the column "pop" from the dataframe

In [ ]:
df.head()  # we can see that "pop" is no longer a column in the dataframe

6: Subset a specific city

I subsetted I city that begins with the same letter as my name (Brodie,B). I chose the city Bucharest

In [ ]:
bucharest = df.query('city == "Bucharest"')  # selects Bucharest in the "city" column

7: Subset the 5 biggest cities in terms of population in the country that Bucharest belongs to (Romania)

This was done in 2 steps. 1st I returned the countries column in the Bucharest dataframe to get Romania. As shown below...

In [ ]:
country = bucharest["country"]  # selects country value from Bucharest subset

Then I wrote a script that would query the top 5 highest population cities for the "pop_M" column that alligned with the country of Romania

In [ ]:
top5_romania = df.query('country == "Romania"').nlargest(5, "pop_M")  # selects 5 largest cities in romania from the "pop_M" column based on population

I then printed the results

In [ ]:
print(top5_romania)  # prints results to show 5 most populated cities in Romania